In [5]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.externals import joblib



df_show = pd.read_excel('data/1029_show.xls',index_col=0)
df_click = pd.read_excel('data/1029_click.xls',index_col=0)
df = pd.merge(df_show, df_click, how='left', on=['_source/uid'])

def f(row):
    if pd.isnull(row['_source/ua_y']):
        val = 0
    else:
        val = 1
    return val

def n(row):
    if pd.isnull(row['_source/outMoney']):
        val = 0
    else:
        val = row['_source/inMoney'] - row['_source/outMoney']
    return val

def m(row):
    if pd.isnull(row['_source/inMoney']):
        val = 0
    else:
        val = row['_source/inMoney']
    return val

def o(row):
    if pd.isnull(row['_source/outMoney']):
        val = 0
    else:
        val = row['_source/outMoney']
    return val

df['target'] = df.apply(f, axis=1)
df['revenue'] = df.apply(n, axis=1)
#df['_source/inMoney'] = df.apply(m, axis=1)
#df['_source/outMoney'] = df.apply(o, axis=1)
df = df[[#'_source/cTime_x','_source/uid',
         #'_source/appkey_x','_source/adSpaceKey_x','_source/advertiserId','_source/advId_x',
         #'_source/gameHtmlId_x','_source/extData/adTriggerType_x',
         #'_source/inMoney',
         #'_source/outMoney',
         'CTR',
         '_source/clientProvince_x',
         #'_source/clientCity_x',
         #'_source/uaBean/osName_x',
         '_source/uaBean/brand_x',
         #'_source/uaBean/model_x','_source/uaBean/browserName_x',
         'target']]

df.to_excel('df_1.xls', encoding='utf-8')
#df['revenue'] = df['_source/outMoney'] - df['_source/inMoney']
#print(df)
#cat_columns = ['_source/uaBean/brand_x']
cat_columns = ['_source/clientProvince_x','_source/uaBean/brand_x']
df = pd.get_dummies(df, prefix_sep="_", columns=cat_columns)

print(df)

df.to_excel('df.xls', encoding='utf-8')



       CTR  target  _source/clientProvince_x_上海  _source/clientProvince_x_中国  \
0    0.002       1                            0                            0   
1    0.002       1                            0                            0   
2    0.002       1                            0                            0   
3    0.002       1                            0                            0   
4    0.002       1                            0                            0   
5    0.030       1                            1                            0   
6    0.002       1                            0                            0   
7    0.002       1                            0                            0   
8    0.002       1                            0                            0   
9    0.002       1                            0                            0   
10   0.001       1                            0                            0   
11   0.002       1                      

In [2]:
import numpy as np
from sklearn.ensemble.gradient_boosting import GradientBoostingClassifier
from sklearn.linear_model.logistic import LogisticRegression
from sklearn.metrics.ranking import roc_auc_score
from sklearn.preprocessing.data import OneHotEncoder

class GradientBoostingWithLR(object):
    def __init__(self):
        self.gbdt_model = None
        self.lr_model = None
        self.gbdt_encoder = None
        self.X_train_leafs = None
        self.X_test_leafs = None
        self.X_trans = None

    def gbdt_train(self, X_train, y_train):
        """定义GBDT模型
        """
        gbdt_model = GradientBoostingClassifier(n_estimators=10, 
                                          max_depth=6, 
                                          verbose=0,
                                          max_features=0.5)
        # 训练学习
        gbdt_model.fit(X_train, y_train)
        return gbdt_model

    def lr_train(self, X_train, y_train):
        """定义LR模型
        """
        lr_model = LogisticRegression()
        lr_model.fit(X_train, y_train)    # 预测及AUC评测
        return lr_model
    

    def gbdt_lr_train(self,X_train, y_train,X_test):
        """训练gbdt+lr模型
        """
        self.gbdt_model = self.gbdt_train(X_train, y_train)

        # 使用GBDT的apply方法对原有特征进行编码
        self.X_train_leafs = self.gbdt_model.apply(X_train)[:,:,0]
        
        # 对特征进行ont-hot编码
        self.gbdt_encoder = OneHotEncoder(categories='auto')
        self.gbdt_encoder.fit(self.X_train_leafs)
        self.X_trans = self.gbdt_encoder.fit_transform(self.X_train_leafs)
        
        #采用LR进行训练
        self.lr_model = self.lr_train(self.X_trans, y_train)
        return self.lr_model
    
    def gbdt_lr_pred(self, model, X_test, y_test):
        """预测及AUC评估
        """
        self.X_test_leafs = self.gbdt_model.apply(X_test)[:,:,0]
        
        (train_rows, cols) =self.X_train_leafs.shape
        X_trans_all = self.gbdt_encoder.fit_transform(np.concatenate((self.X_train_leafs, self.X_test_leafs), axis=0))
        
        y_pred = model.predict_proba(X_trans_all[train_rows:])[:, 1]
        auc_score = roc_auc_score(y_test, y_pred)
        print('GBDT+LR AUC score: %.5f' % auc_score)
        return auc_score
    
    def model_assessment(self, model, X_test, y_test, model_name="GBDT"):
        """模型评估
        """
        y_pred = model.predict_proba(X_test)[:,1]
        auc_score = roc_auc_score(y_test, y_pred)
        print("%s AUC score: %.5f" % (model_name,auc_score))
        return auc_score

In [3]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split

def load_data():
    """
    调用sklearn的iris数据集，将多类数据构造成2分类数据,同时切分训练测试数据集
    """
    #iris_data = load_iris()
    #print(type(iris_data))
    #print(iris_data)
    #X = iris_data['data']
    #y = iris_data['target'] == 2
    
    #print(X.shape)
    #print(y.shape)
    #X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=0)
    #print(X_train.shape)
    #print(X_test.shape)
    #print(y_train.shape)
    #print(y_test.shape)
    
    print(X.shape)
    print(y.shape)

    print(X_train.shape)
    print(X_test.shape)
    print(y_train.shape)
    print(y_test.shape)
    
    print(X_train)
    print(y_train)
    
    return X_train,X_test,y_train, y_test


    
X = df.drop("target", axis = 1)
y = df['target'].values
y = y.reshape(-1, 1)
X_train, X_test, y_train, y_test = train_test_split(X, y,test_size = 0.3, random_state=0)

        
gblr = GradientBoostingWithLR()
gbdt_lr_model = gblr.gbdt_lr_train(X_train, y_train, X_test)
gblr.model_assessment(gblr.gbdt_model, X_test, y_test)
gblr.gbdt_lr_pred(gbdt_lr_model, X_test, y_test)



GBDT AUC score: 0.62864
GBDT+LR AUC score: 0.57524


/Users/baicongmin/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/baicongmin/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/baicongmin/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.5752427184466019